データ分析やモデリングを行う過程で、かなりの時間がデータの前処理に使われる  
データの前処理とは、読み込みやクリーニング、変形、整形といった作業のことで、分析者の時間の80％以上を占めるとも言われている  
その理由としては、扱いたいデータがファイルやデータベースに保存されていても、行いたい作業にとって適切な形式で保存されていないことがあるためである  
そのような場合に多くの分析者は、汎用的なプログラミング言語を用いて、データを別の形式に変換するためのアドホックな処理を行うという対応を取る  
ここで使われるプログラミング言語としては、Python、Perl、R、Java、Unixのテキスト処理ツールであるsedやawkなどがある  
pandas本体や、pandasから使えるPython組み込みの機能では、高度で柔軟性も高く、高速なツールセットが提供されているので、それらを用いてデータを適切な形に変換する作業ができる  

# 7.1  欠損値の取り扱い

pandasは欠損値の扱いをできる限り面倒くさくないようにするというのが1つの目的で、例えば、記述統計量の算出時にはすべての欠損値が除外される  
pandasオブジェクトでの欠損値の表現方法は、すべての事例を完璧に取り扱えるものではないが、多くの場合には十分な機能を持っている  
数値データについては、pnadasは欠損値の表現として浮動小数値NaNを使う  
このNaNを欠損値を見つけるための標識と呼ぶ

In [1]:
import pandas as pd
import numpy as np

In [2]:
string_data = pd.Series(["aardvark", "artichoke", np.nan, "avocado"])
string_data

0     aardvark
1    artichoke
2          NaN
3      avocado
dtype: object

In [3]:
string_data.isnull()

0    False
1    False
2     True
3    False
dtype: bool

R言語にならって、pandasでは欠損値をNAと呼ぶ  
統計アプリケーションにおいてNAというデータは、存在しないデータを指す場合と、存在するが観測できなかったデータを指す場合がある  
分析のためにデータをクリーニングする過程では、欠損値の数を数えるなど、欠損値自体の分析をする必要が出てくる  
その目的は様々で、データの収取時に問題が起きていないかという観点で確認する場合もあれば。欠損値によってデータ内に偏りが発生していないかという観点で確認することもある  

In [4]:
string_data[0] = None

In [5]:
string_data.isnull()

0     True
1    False
2     True
3    False
dtype: bool

Python組み込みの値Noneがオブジェクトに含まれている場合も欠損値として扱われる

欠損値の取り扱い方法については、内部実装を改善する取り組みが進行中だが、現状でもユーザー向けにAPIが提供されているisnullなどの関数は、内部実装の問題の多くを抽象化して隠してくれる  

欠損値を扱うメソッド

|メソッド|説明|
|:-|:-|
|dropna|指定した幅について、その軸のラベルの値に欠損値が含まれる場合はラベルを削除する。含まれる欠損値の数がいくつまでなら削除しないか、閾値で指定することもできる|
|fillna|欠損値を指定した値で穴埋めする。または、"ffill"や"bfill"などの指定した方法で穴埋めする|
|isnull|各値が欠損値であるかを示す一連の真偽値を戻す|
|notnull|isnullの反対の動作をする|

## 7.1.1  欠損値を削除する

欠損値を削除する方法はいくつかあり、isnullの真偽値の配列を用いて手動で削除する方法もあるが、dropnaを使う方法が便利である  

In [6]:
from numpy import nan as NA

In [7]:
data = pd.Series([1, NA, 3.5, NA, 7])
data

0    1.0
1    NaN
2    3.5
3    NaN
4    7.0
dtype: float64

In [8]:
data.dropna()

0    1.0
2    3.5
4    7.0
dtype: float64

In [9]:
data[data.notnull()]

0    1.0
2    3.5
4    7.0
dtype: float64

シリーズに対してdropnaメソッドを用いると、欠損値でないデータとそのインデックスを持ったシリーズが返される  
これは、notnullを選択して返すのと同じ結果が返ってくる

In [10]:
data = pd.DataFrame([[1., 6.5, 3.], [1., NA, NA], [NA, NA, NA], [NA, 6.5, 3.]])
data

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


In [11]:
cleaned = data.dropna()
cleaned

,0,1,2
0,1.0,6.5,3.0


データフレームオブジェクトの場合は、すべてのデータが欠損値である行か列を削除したいか、欠損値を1つでも含む行か列を削除したいかのどちらかのことが多い  
dropnaメソッドはデフォルトでは、欠損値を1つでも含む行をすべて削除する

In [12]:
data.dropna(how="all")

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
3,NaN,6.5,3.0


dropnaのキーワード引数のhowにallを指定すると、すべてのデータが欠損値である行のみが削除される

In [13]:
data[4] = NA
data

,0,1,2,4
0,1.0,6.5,3.0,NaN
1,1.0,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,NaN,6.5,3.0,NaN


In [14]:
data.dropna(axis=1, how="all")

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


dropnaで行ではなく列を削除する場合は、axis=1を指定することでできる

In [15]:
df = pd.DataFrame(np.random.randn(7, 3))
df.iloc[:4, 1] = NA
df.iloc[:2, 2] = NA
df

,0,1,2
0,0.035116,NaN,NaN
1,2.099262,NaN,NaN
2,0.240307,NaN,1.020132
3,-0.454577,NaN,1.193290
4,0.203512,-0.701829,1.693891
5,-0.067693,0.817701,-1.070568
6,0.276790,-1.002371,2.690998


In [16]:
df.dropna()

,0,1,2
4,0.203512,-0.701829,1.693891
5,-0.067693,0.817701,-1.070568
6,0.276790,-1.002371,2.690998


In [17]:
df.dropna(thresh=2)

,0,1,2
2,0.240307,NaN,1.020132
3,-0.454577,NaN,1.193290
4,0.203512,-0.701829,1.693891
5,-0.067693,0.817701,-1.070568
6,0.276790,-1.002371,2.690998


一定数の観測値(欠損値ではない値)が含まれる行だけを残したい場合は、キーワード引数のthreshに整数を渡すことでできる

## 7.1.2  欠損値を穴埋めする

欠損値を削除するのではなく、欠損値を何らかの方法で埋めることもできる  
たいていの場合、その役目を果たしてくれるのがfillnaメソッドになる

In [18]:
df.fillna(0)

,0,1,2
0,0.035116,0.000000,0.000000
1,2.099262,0.000000,0.000000
2,0.240307,0.000000,1.020132
3,-0.454577,0.000000,1.193290
4,0.203512,-0.701829,1.693891
5,-0.067693,0.817701,-1.070568
6,0.276790,-1.002371,2.690998


fillnaメソッドに何らかの値を引数として与えて呼び出すと、その値で欠損値を置き換えることができる

In [19]:
df.fillna({1: 0.5, 2: 0})

,0,1,2
0,0.035116,0.500000,0.000000
1,2.099262,0.500000,0.000000
2,0.240307,0.500000,1.020132
3,-0.454577,0.500000,1.193290
4,0.203512,-0.701829,1.693891
5,-0.067693,0.817701,-1.070568
6,0.276790,-1.002371,2.690998


fillnaメソッドにキーに列名、バリューに変更したい値のディクショナリを渡すことで、列ごとに異なる値で埋めることができる

In [20]:
df.fillna(0, inplace=True)
df

,0,1,2
0,0.035116,0.000000,0.000000
1,2.099262,0.000000,0.000000
2,0.240307,0.000000,1.020132
3,-0.454577,0.000000,1.193290
4,0.203512,-0.701829,1.693891
5,-0.067693,0.817701,-1.070568
6,0.276790,-1.002371,2.690998


fillnaメソッドはデフォルトでは新しいオブジェクトを戻しますが、キーワード引数のinplaceにTrueを渡すことで既存のオブジェクトを直接変更することもできる

In [21]:
df = pd.DataFrame(np.random.randn(6, 3))
df.iloc[2:, 1] = NA
df.iloc[4:, 2] = NA
df

,0,1,2
0,0.245041,-0.330094,0.646930
1,-2.143793,-0.359253,-1.073825
2,0.555019,NaN,-0.415314
3,2.478872,NaN,-0.069876
4,1.284074,NaN,NaN
5,2.446610,NaN,NaN


In [22]:
df.fillna(method="ffill", limit=2)

,0,1,2
0,0.245041,-0.330094,0.646930
1,-2.143793,-0.359253,-1.073825
2,0.555019,-0.359253,-0.415314
3,2.478872,-0.359253,-0.069876
4,1.284074,NaN,-0.069876
5,2.446610,NaN,-0.069876


再インデックス付けのときのように、fillnaでキーワード引数のmethodにffillを指定することで前の値で穴埋めすることもできる  
limitに整数を渡して、穴埋めする上限数を指定している

In [23]:
data = pd.Series([1., NA, 3.5, NA, 7])
data.fillna(data.mean())

0    1.000000
1    3.833333
2    3.500000
3    3.833333
4    7.000000
dtype: float64

上のように工夫をすることで平均値で埋めたり、中央値で埋めたりすることもできる

メソッドの引数

|引数|説明|
|:-|:-|
|value|欠損値の穴埋めに用いられるスカラー値。またはディクショナリ系のオブジェクト|
|method|穴埋め方法を指定する。ほかの引数が何も指定されなかった場合、デフォルトではffillが適用される|
|axis|穴埋めをしたい軸。デフォルトでは0(行方向)|
|inplace|コピーを作るのではなく、呼び出し元のオブジェクトを直接変更する|
|limit|前方(method="ffill")、後方(method="bfill")への穴埋めのときに、連続した穴埋めを最大何回行うか|

# 7.2  データの変形

## 7.2.1  重複の除去

In [24]:
data = pd.DataFrame({"k1": ["one", "two"] * 3 + ["two"],
                     "k2": [1, 1, 2, 3, 3, 4, 4]})
data

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4
6,two,4


In [25]:
data.duplicated()

0    False
1    False
2    False
3    False
4    False
5    False
6     True
dtype: bool

データフレームのduplicatedメソッドは、それぞれの行が重複しているかどうか(同じ値を持つ行が前にもあるか)を示した、真偽値のシリーズを返す

In [26]:
data.drop_duplicates()

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4


関連するメソッドにdrop_duplicatesメソッドがあり、このメソッドは重複した行を削除したデータフレームを返す

In [27]:
data["v1"] = range(7)
data

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
5,two,4,5
6,two,4,6


In [28]:
data.drop_duplicates(["k1"])

,k1,k2,v1
0,one,1,0
1,two,1,1


duplicatedやdrop_duplicatesメソッドは、デフォルトでは全ての列が同じ値の場合に重複と判定するが、列名のリストを渡すことで対象を一部の列に限定することができる

In [29]:
data.drop_duplicates(["k1", "k2"], keep="last")

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
6,two,4,6


duplicatedやdrop_duplicatesメソッドは、デフォルトでは重複が見つかった場合に最初の値は重複とみなさないが、キーワード引数のkeepにlastを渡すと最後の値を重複とみなさないようにすることができる

## 7.2.2  関数やマッピングを用いたデータの変換

さまざまなデータセットを扱っていると、配列やシリーズ、データフレーム内の列の値に基づいて変換を行いたいことがある  

In [30]:
data = pd.DataFrame({"food": ["bacon", "pulled pork", "bacon", "Pastrami", "corned beef", "Bacon", "pastrami", "honey ham", "nova lox"],
                     "ounces": [4, 3, 12, 6, 7.5, 8, 3, 5, 6]})
data

,food,ounces
0,bacon,4.0
1,pulled pork,3.0
2,bacon,12.0
3,Pastrami,6.0
4,corned beef,7.5
5,Bacon,8.0
6,pastrami,3.0
7,honey ham,5.0
8,nova lox,6.0


上のデータに含まれる要素に応じた新しいデータの列を追加したい

In [31]:
meat_to_animal = {"bacon": "pig",
                  "pulled pork": "pig",
                  "pastrami": "cow",
                  "corned beef": "cow",
                  "honey ham": "pig",
                  "nova lox": "salmon"}

新しい列を追加するために要素の名前にマッピングするデータをディクショナリとして作成する

In [32]:
lowercased = data["food"].str.lower()
lowercased

0          bacon
1    pulled pork
2          bacon
3       pastrami
4    corned beef
5          bacon
6       pastrami
7      honey ham
8       nova lox
Name: food, dtype: object

In [33]:
lowercased.map(meat_to_animal)

0       pig
1       pig
2       pig
3       cow
4       cow
5       pig
6       cow
7       pig
8    salmon
Name: food, dtype: object

In [34]:
data["animal"] = lowercased.map(meat_to_animal)
data

,food,ounces,animal
0,bacon,4.0,pig
1,pulled pork,3.0,pig
2,bacon,12.0,pig
3,Pastrami,6.0,cow
4,corned beef,7.5,cow
5,Bacon,8.0,pig
6,pastrami,3.0,cow
7,honey ham,5.0,pig
8,nova lox,6.0,salmon


シリーズのmapメソッドは、マッピングを定義した関数オブジェクトかディクショナリ系オブジェクトを渡すことで、対応したシリーズにして返してくれる  
そのままだと大文字と小文字を区別してしまうのでstr.lowerメソッドを使って要素をすべて小文字にしてから、マッピングをしている

In [35]:
data["food"].map(lambda x: meat_to_animal[x.lower()])

0       pig
1       pig
2       pig
3       cow
4       cow
5       pig
6       cow
7       pig
8    salmon
Name: food, dtype: object

シリーズのmapメソッドに関数を渡す場合は上のようにすることでできる  
mapメソッドは、要素ごとの変換や、その他のデータクリーニング操作を行うのに便利な手段になる

## 7.2.3  値の置き換え

fillnaメソッドを用いた欠損値の穴埋めは、一般的な値の置き換え操作の特殊な例と考えることができる  
先程紹介したmapメソッドを用いることでオブジェクト内の一部の値を修正することができる  
しかし、もっとシンプルで柔軟性が高いのがreplaceを使う方法になる

In [36]:
data = pd.Series([1., -999., 2., -999., -1000., 3.])
data

0       1.0
1    -999.0
2       2.0
3    -999.0
4   -1000.0
5       3.0
dtype: float64

上のようなデータがあるとして、おそらく-999というのは欠損値を示す標識と考えられる  

In [37]:
data.replace(-999, np.nan)

0       1.0
1       NaN
2       2.0
3       NaN
4   -1000.0
5       3.0
dtype: float64

replaceメソッドを使って、変更前と後の値を渡すことで、変更した新しいシリーズを返してくれる

In [38]:
data.replace([-999, -1000], np.nan)

0    1.0
1    NaN
2    2.0
3    NaN
4    NaN
5    3.0
dtype: float64

複数の値を同時に変更したい場合は、第一引数にリストとして渡すことでできる

In [39]:
data.replace([-999, -1000], [np.nan, 0])

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

複数の値を別々の値に置き換えたいときは、第一引数にリストを渡したうえで、要素数が同数のリストを第二引数を渡すことでできる

In [40]:
data.replace({-999: np.nan, -1000: 0})

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

また、replaceメソッドはディクショナリとして渡すこともできる

## 7.2.4  軸のインデックスの名前を変更する

シリーズの各値だけでなく、軸のラベルも変換できる  
軸のラベルの変換にも、異なるラベルを持った新たなオブジェクトを生成するための、ある種の関数やマッピングを使用する  
また、新たなデータを構造を作成せずに、軸を変更することも可能である

In [41]:
data = pd.DataFrame(np.arange(12).reshape((3, 4)),
                    index=["Ohio", "Colorado", "New York"],
                    columns=["one", "two", "three", "four"])
data

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
New York,8,9,10,11


In [42]:
data.index.map(lambda x: x[:4].upper())

Index(['OHIO', 'COLO', 'NEW '], dtype='object')

シリーズと同様に、軸のインデックスオブジェクトにもmapメソッドがあり、新たな軸のインデックスオブジェクトを生成して返してくれる

In [43]:
data.index = data.index.map(lambda x: x[:4].upper())
data

,one,two,three,four
OHIO,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


mapで変換して生成したインデックスオブジェクトを変更したいデータフレームのインデックスオブジェクトに代入することで変更することができる

In [44]:
data.rename(index=str.title, columns=str.upper)

,ONE,TWO,THREE,FOUR
Ohio,0,1,2,3
Colo,4,5,6,7
New,8,9,10,11


元のデータフレームを変更せずに、変換後の軸を持つデータフレームを別に作成したい場合にはrenameメソッドが使える  
renameメソッドのキーワード引数indexとcolumnsに関数を渡すことでラベルの名前を変えて新しいデータフレームを返してくれる

In [45]:
data.rename(index={"Ohio": "INDIANA"},
            columns={"three": "peekaboo"})

,one,two,peekaboo,four
OHIO,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


renameメソッドのキーワード引数indexとcolumnsにはディクショナリ系オブジェクトも渡すことができ、一部の軸のラベルのみを変更することができる

In [46]:
data.rename(index={"OHIO": "INDIANA"}, inplace=True)
data

,one,two,three,four
INDIANA,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


renameメソッドを使って新しいデータフレームを作るのではなく、元のデータフレームに直接変更を加えたい場合は、inplaceにTrueを渡すことでできる

## 7.2.5  離散化とビニング

連続化したデータを離散化したり、分析のために「ビン」に分割したいときがある  

In [47]:
ages = [20, 22, 25, 27, 21, 23, 37, 31, 61, 45, 41, 32]

このデータを18から25、26から35、36から60、61以上の4つのビンに分割する

In [48]:
bins = [18, 25, 35, 60, 100]
cats = pd.cut(ages, bins)
cats

[(18, 25], (18, 25], (18, 25], (25, 35], (18, 25], ..., (25, 35], (60, 100], (35, 60], (35, 60], (25, 35]]
Length: 12
Categories (4, interval[int64]): [(18, 25] < (25, 35] < (35, 60] < (60, 100]]

In [49]:
type(cats)

pandas.core.arrays.categorical.Categorical

In [50]:
print(cats[4])

(18, 25]


pandasのcut関数を使うことで、第一引数に渡されたデータ(リスト)が第二引数に渡された基準になるデータ(リスト)のどこに入るのかという情報などを持った特殊なカテゴリ型オブジェクトをを返す  
このオブジェクトは、ビンの名前を示す文字列の配列と同じように扱うことができる

In [51]:
cats.codes

array([0, 0, 0, 1, 0, 0, 2, 1, 3, 2, 2, 1], dtype=int8)

In [52]:
cats.categories

IntervalIndex([(18, 25], (25, 35], (35, 60], (60, 100]],
              closed='right',
              dtype='interval[int64]')

カテゴリ型オブジェクトはcode属性を持っていて、元のデータがビンのどこに当てはまるのかという情報を持っている  
また、categories属性には各カテゴリの名前を指定するための情報を持っている

In [53]:
pd.value_counts(cats)

(18, 25]     5
(35, 60]     3
(25, 35]     3
(60, 100]    1
dtype: int64

value_counts関数(メソッドでも可)を使うことで各ビンの要素の合計数をシリーズにして返してくれる

カテゴリ型オブジェクトで使われている(と\[は、数学での表記法と同じで、(が開区間(境界を含まない)で[が閉区間(境界を含む)であることを意味する

In [54]:
pd.cut(ages, [18, 26, 36, 61, 100], right=False)

[[18, 26), [18, 26), [18, 26), [26, 36), [18, 26), ..., [26, 36), [61, 100), [36, 61), [36, 61), [26, 36)]
Length: 12
Categories (4, interval[int64]): [[18, 26) < [26, 36) < [36, 61) < [61, 100)]

cut関数を使うときにキーワード引数のrightにFalseを渡すことで、閉区間を左にして開区間を右にすることができる  

In [55]:
group_names = ["Youth", "YoungAdult", "MiddleAges", "Senior"]

In [56]:
pd.cut(ages, bins, labels=group_names)

[Youth, Youth, Youth, YoungAdult, Youth, ..., YoungAdult, Senior, MiddleAges, MiddleAges, YoungAdult]
Length: 12
Categories (4, object): [Youth < YoungAdult < MiddleAges < Senior]

cut関数のキーワード引数のlabelsにビンの名前をリストか配列として渡すと、ビンの名前を数字の範囲ではなく定めた名前にすることができる

In [57]:
data = np.random.rand(20)
data

array([0.27255551, 0.66072465, 0.14031901, 0.56763531, 0.68896862,
       0.31667683, 0.48848723, 0.89271927, 0.55444026, 0.24194993,
       0.72890278, 0.48521276, 0.02889393, 0.04545286, 0.18125178,
       0.18475407, 0.14927488, 0.45176203, 0.69047427, 0.80594902])

In [58]:
pd.cut(data, 4, precision=2)

[(0.24, 0.46], (0.46, 0.68], (0.028, 0.24], (0.46, 0.68], (0.68, 0.89], ..., (0.028, 0.24], (0.028, 0.24], (0.24, 0.46], (0.68, 0.89], (0.68, 0.89]]
Length: 20
Categories (4, interval[float64]): [(0.028, 0.24] < (0.24, 0.46] < (0.46, 0.68] < (0.68, 0.89]]

pandasのcut関数の第二引数でビンの境界をリストや配列で明示的に指定する以外に、ビンの数を整数値で渡すこともできる  
その場合、データの最小値と最大値を基にして、その間を等間隔で区切ったビンが設定される  
上で同時にキーワード引数で指定している precisionはビンを設定する際に境界線を小数点以下の何桁の精度で実行するかを決まることができ、上の場合小数点以下第2の制度で実行している

In [59]:
data = np.random.randn(1000)

In [60]:
cats = pd.qcut(data, 4)
cats

[(0.00305, 0.681], (0.681, 3.435], (0.00305, 0.681], (0.00305, 0.681], (-0.63, 0.00305], ..., (-2.737, -0.63], (-0.63, 0.00305], (0.00305, 0.681], (0.681, 3.435], (0.681, 3.435]]
Length: 1000
Categories (4, interval[float64]): [(-2.737, -0.63] < (-0.63, 0.00305] < (0.00305, 0.681] < (0.681, 3.435]]

In [61]:
pd.value_counts(cats)

(0.681, 3.435]      250
(0.00305, 0.681]    250
(-0.63, 0.00305]    250
(-2.737, -0.63]     250
dtype: int64

cut関数に似た関数がpandasにもう一つあり、qcut関数というものがある  
cut関数はそれぞれのビンのデータ数が同じにはならないが、qcut関数は指定された数にデータ数を平等に分けるように区切るためのカテゴリ型オブジェクトを返す

In [62]:
a = pd.qcut(data, [0., 0.1, 0.5, 0.9, 1])

In [63]:
a.value_counts()

(-2.737, -1.256]     100
(-1.256, 0.00305]    400
(0.00305, 1.349]     400
(1.349, 3.435]       100
dtype: int64

qcut関数の第二引数に整数ではなく、0から1までの数で構成されたリストや配列を渡すことで、指定の割合ごとにデータ数を分けるためのカテゴリ型オブジェクトを作成することができる  
上の場合、10%,40%,40%,10%の割合で区切るためのカテゴリ型を返す

## 7.2.6  外れ値の検出と除去

In [64]:
import pandas as pd
import numpy as np

外れ値の除去や変換は、ほとんどが配列に対する操作で対応できる問題になる  

In [65]:
data = pd.DataFrame(np.random.randn(1000, 4))
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,0.006317,-0.008964,-0.032814,-0.052506
std,0.990079,0.974417,0.964417,0.992320
min,-3.808079,-3.238836,-3.993755,-3.217842
25%,-0.694782,-0.702695,-0.677930,-0.759578
50%,-0.022383,-0.012505,-0.044186,-0.055845
75%,0.661694,0.658612,0.606714,0.633502
max,3.300260,3.100614,2.714235,3.466208


以下ではいくつかの正規分布をするデータの入ったデータフレームを使って考える

In [66]:
col = data[2]

In [67]:
col[np.abs(col) > 3]

763   -3.993755
Name: 2, dtype: float64

上では、4つの列のうち1つの列で外れ値を見つけている  
外れ値としては絶対値が3より大きなものとして選択している

In [68]:
data[(np.abs(data) > 3).any(1)]

,0,1,2,3
93,3.030244,0.660172,-1.486659,0.051502
218,0.697720,3.100614,2.714235,1.628832
257,0.683119,-3.238836,1.228489,1.207106
353,3.300260,-1.135044,-0.209802,0.882911
463,-0.633247,1.152594,1.123051,3.055179
560,-0.826214,-1.490126,0.876623,-3.217842
571,-0.586784,0.278118,-0.687450,3.135653
672,-3.808079,-0.446155,0.185400,-0.190820
752,-0.577440,-0.196665,-0.446540,3.466208
763,0.146664,0.639894,-3.993755,0.338811


外れ値が行に1つ以上ある行を選択するには、真偽値のデータフレームに対してanyメソッドを使うことでその行や列を選択できる  
anyに渡している1は行ごとにするためのaxis=1という意味で、デフォルトだと列ごとに真偽値のTrueが1つでもあれば選択する(列をインデックスとした真偽値のシリーズを返す)ようになっている

In [69]:
data[np.abs(data) > 3] = np.sign(data) * 3
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,0.006795,-0.008826,-0.031820,-0.052945
std,0.986238,0.973337,0.960837,0.989501
min,-3.000000,-3.000000,-3.000000,-3.000000
25%,-0.694782,-0.702695,-0.677930,-0.759578
50%,-0.022383,-0.012505,-0.044186,-0.055845
75%,0.661694,0.658612,0.606714,0.633502
max,3.000000,3.000000,2.714235,3.000000


外れ値を選択して代入を行うことで新しい値を設定することもできる  
上のsign関数は正の数か負の数かに応じて1か-1を返す関数になる(それに3をかけているので3か-3を数に応じて返す)  
sign関数の動作は下を参照

In [70]:
np.sign(data).head()

,0,1,2,3
0,-1.0,-1.0,-1.0,1.0
1,-1.0,1.0,1.0,1.0
2,1.0,-1.0,1.0,1.0
3,1.0,-1.0,-1.0,1.0
4,1.0,-1.0,-1.0,-1.0


## 7.2.7  順列(ランダムな並べ替え)やランダムサンプリング

In [71]:
sampler = np.random.permutation(5)
sampler

array([1, 4, 3, 2, 0])

numpyのrandom.permutation関数に並べ替えたい軸の長さを引数として渡すと、ランダムな順番で並んだ整数の配列を得ることができる  
この配列を使って列や行の順列を得る(ランダムに並べ替える)ことができる

In [72]:
df = pd.DataFrame(np.arange(5 * 4).reshape((5, 4)))
df

,0,1,2,3
0,0,1,2,3
1,4,5,6,7
2,8,9,10,11
3,12,13,14,15
4,16,17,18,19


In [73]:
df.take(sampler)

,0,1,2,3
1,4,5,6,7
4,16,17,18,19
3,12,13,14,15
2,8,9,10,11
0,0,1,2,3


random.permutation関数を用いて得られた配列をilocを用いたインデックス参照や、それと同等のことを行うtake関数(軸の順番を指定された順番に並べる関数)に渡すことで使用できる  

In [74]:
df.sample(n=3)

,0,1,2,3
0,0,1,2,3
4,16,17,18,19
1,4,5,6,7


ランダムに指定した行数だけ非復元抽出(同じデータが選ばれないように抽出する方法)で選択するには、シリーズやデータフレームに対してsampleメソッドを使って取り出したい行数を指定することでできる(第一引数、もしくはキーワード引数のnが抽出数の指定)

In [75]:
choices = pd.Series([5, 7, -1, 6, 4])
choices.sample(n=10, replace=True)

2   -1
3    6
0    5
1    7
0    5
0    5
0    5
1    7
4    4
3    6
dtype: int64

復元抽出(同じデータが再度選ばれ可能性のある抽出方法)で選択するには、sampleメソッドのキーワード引数replaceにTrueを渡すことでできる

## 7.2.8  標識変数やダミー変数の計算

統計モデリングや機械学習アプリケーションでよく用いる変換方法の1つに、カテゴリ変素から「ダミー変数」や「標識変数」の行列への変換がある  
たとえば、データフレーム内の特定の列にk個の独立な値がある場合は、0か1の値を持つk個の列を持った行列やデータフレームへと変換することになる  
自前で実装することも難しくはないが、このような変数のために、pandasにはget_dummies関数がある

In [76]:
df = pd.DataFrame({"key": ["b", "b", "a", "c", "a", "b"],
                    "data1": range(6)})
df

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,b,5


In [77]:
pd.get_dummies(df["key"])

,a,b,c
0,0,1,0
1,0,1,0
2,1,0,0
3,0,0,1
4,1,0,0
5,0,1,0


pandasのget_dummies関数に列を指定したシリーズを渡すことで、標識変数のデータフレームを作成して返してくれる

In [78]:
dummies = pd.get_dummies(df["key"], prefix="key")

In [79]:
dummies

,key_a,key_b,key_c
0,0,1,0
1,0,1,0
2,1,0,0
3,0,0,1
4,1,0,0
5,0,1,0


In [80]:
df_with_dummy = df[["data1"]].join(dummies)
df_with_dummy

,data1,key_a,key_b,key_c
0,0,0,1,0
1,1,0,1,0
2,2,1,0,0
3,3,0,0,1
4,4,1,0,0
5,5,0,1,0


標識変数のデータフレームの列名にプレフィックス(接頭語)を付けたい場合は、get_dummies関数のキーワード引数prefixに付けたいプレフィックスを渡すことでできる  
また、データフレームにjoinメソッドを使ってデータフレームを渡すことでデータフレーム同士を結合することができる

In [81]:
mnames = ["movie_id", "title", "genres"]
movies = pd.read_table(r".\download_file\datasets\movielens\movies.dat", sep="::", header=None, names=mnames)
movies[:10]

C:\Users\kumak\anaconda3\lib\site-packages\ipykernel_launcher.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  


,movie_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children's
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


データフレーム内の特定の行に複数のカテゴリにまたがる情報が含まれている場合は少し複雑なことになる  
上のデータのgenres行を基にして各ジャンルの標識変数の行を追加するには、データを少し改変する必要がある  

In [82]:
all_genres = []
for x in movies.genres:
    all_genres.extend(x.split("|"))
genres = pd.unique(all_genres)
genres

array(['Animation', "Children's", 'Comedy', 'Adventure', 'Fantasy',
       'Romance', 'Drama', 'Action', 'Crime', 'Thriller', 'Horror',
       'Sci-Fi', 'Documentary', 'War', 'Musical', 'Mystery', 'Film-Noir',
       'Western'], dtype=object)

まずはgenres行に含まれる要素をすべて1つずつに区切ったうえでリストに追加し、ユニークなジャンルの一覧を取り出してみる

In [83]:
zero_matrix = np.zeros((len(movies), len(genres)))
dummies = pd.DataFrame(zero_matrix, columns=genres)

In [84]:
dummies.head()

,Animation,Children's,Comedy,Adventure,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Sci-Fi,Documentary,War,Musical,Mystery,Film-Noir,Western
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


標識変数のデータフレームを作成する1つの方法として、すべてのセルをゼロで埋めたデータフレームを作成して使う方法がある  
そのために0で埋められた、行数がmovies、列数がgenresの長さでできたデータフレームを作成した

In [85]:
gen = movies.genres[0]
gen.split("|")

['Animation', "Children's", 'Comedy']

In [86]:
dummies.columns.get_indexer(gen.split("|"))

array([0, 1, 2], dtype=int64)

moviesのgenres列の要素からひとつづつ抽出し、dummiesの対応する行、およびジャンルの値を1に設定していきたい  
そのために、columnsオブジェクトのget_indexerメソッドを使って、あてはまる列のインデックスが得られるかを上で試してみている  
うまくいきそうなのでループしてみる

In [87]:
for i, gen in enumerate(movies.genres):
    indices = dummies.columns.get_indexer(gen.split("|"))
    dummies.iloc[i, indices] = 1

In [88]:
dummies.head()

,Animation,Children's,Comedy,Adventure,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Sci-Fi,Documentary,War,Musical,Mystery,Film-Noir,Western
0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


得られた列のインデックスを使って、ilocでデータフレームのセルを指定して1を代入していくことで、標識変数を作成できた

In [89]:
movies_windic = movies.join(dummies.add_prefix("Genre_"))
movies_windic.head()

,movie_id,title,genres,Genre_Animation,Genre_Children's,Genre_Comedy,Genre_Adventure,Genre_Fantasy,Genre_Romance,Genre_Drama,...,Genre_Crime,Genre_Thriller,Genre_Horror,Genre_Sci-Fi,Genre_Documentary,Genre_War,Genre_Musical,Genre_Mystery,Genre_Film-Noir,Genre_Western
0,1,Toy Story (1995),Animation|Children's|Comedy,1.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,Jumanji (1995),Adventure|Children's|Fantasy,0.0,1.0,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,Grumpier Old Men (1995),Comedy|Romance,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,Waiting to Exhale (1995),Comedy|Drama,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,Father of the Bride Part II (1995),Comedy,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


作成した標識変数を基のデータフレームと結合すると上のようになる

In [90]:
values = np.random.rand(10)
values

array([0.59971223, 0.6547844 , 0.54570467, 0.38497957, 0.46887005,
       0.79100168, 0.72582161, 0.89126974, 0.71306277, 0.20628599])

In [91]:
bins = [0, 0.2, 0.4, 0.6, 0.8, 1]
pd.get_dummies(pd.cut(values, bins))

,"(0.0, 0.2]","(0.2, 0.4]","(0.4, 0.6]","(0.6, 0.8]","(0.8, 1.0]"
0,0,0,1,0,0
1,0,0,0,1,0
2,0,0,1,0,0
3,0,1,0,0,0
4,0,0,1,0,0
5,0,0,0,1,0
6,0,0,0,1,0
7,0,0,0,0,1
8,0,0,0,1,0
9,0,1,0,0,0


統計処理において役に立つレシピとして、cutなどの離散化関数とget_dummiesを組み合わせることで、要素がどの列に属しているかを表す標識変数のデータフレームを作成することができる

# 7.3  文字列操作

Pythonは昔から、生データ操作用の言語として人気があり、その理由の1つは、文字列やテキストデータを処理しやすい点にある  
ほとんどの文字列操作は、文字列オブジェクト組み込みのメソッドで簡単に行える  
もっと複雑なパターンマッチングやテキスト操作には、正規表現が必要になることがある  
Python組み込みの文字列操作メソッドや正規表現の機能にpandasが加わると、配列のすべてのデータに対して文字列操作や正規表現を簡単に適用できるようになり、更に欠損値という悩みの種をより扱うことも可能になる

## 7.3.1  文字列オブジェクトのメソッド

文字列操作をしたり、その処理をスクリプト化したりする場合、大抵は、組み込みの文字列処理メソッドだけで充分処理できる

In [92]:
val = "a,b, guido"
val.split(",")

['a', 'b', ' guido']

何かしらの文字列で区切られている場合はsplitメソッドを使うことで分割できる

In [93]:
pieces = [x.strip() for x in val.split(",")]
pieces

['a', 'b', 'guido']

splitで分割する際に、文字列の前後の空白文字を取り除くstripメソッドも一緒に用いられることが多い

In [94]:
first, second, third = pieces
first + "::" + second + "::" + third

'a::b::guido'

文字列を+で連結することも可能

In [95]:
"::".join(pieces)

'a::b::guido'

特定の区切り文字で連結する場合はjoinを使うとよい

In [96]:
"guido" in val

True

In [97]:
val.index(",")

1

In [98]:
val.find(":")

-1

文字列内で部分文字列を見つけるにはキーワードのinを使う
indexとfindを使ってもでき、インデックス値を返してくれる  
indexとfindの違い、見つからなかったときに例外を起こすか-1を返すかの違いになる

In [99]:
val.count(",")

2

countメソッドは、指定した部分文字列が見つかった回数を返す

In [100]:
val.replace(",", "::")

'a::b:: guido'

In [101]:
val.replace(",", "")

'ab guido'

replaceメソッドは、あるパターンが見つかったときに他の文字列に置き換える  
置き換え後の文字列に空白を渡せば特定のパターンを消すこともできる

Pythonの組み込み文字列メソッド

|メソッド|説明|
|:-|:-|
|count|指定した部分文字列が文字列内に重複せずに見つかった回数を回数を戻す|
|endswith|文字列が、指定したサフィックスで終わっている場合にTrueを返す|
|startswith|文字列が、指定したプレフィックスで始まっている場合にTrueを返す|
|join|文字列を区切り文字として、指定した指定したリストやタプルに含まれる文字列を連結する|
|index|文字列内で、指定した部分文字列が見つかったときの、部分文字列の先頭の文字の位置を返す|
|find|文字列内で、指定した部分文字列が最初に見つかったときの、部分文字列の先頭の文字の位置を返す。見つからなかったときは-1を返す|
|rfind|文字列内で、指定した部分文字列が最後に見つかったときの、部分文字列の先頭の文字の位置を返す。見つからなかったときは-1を返す|
|replace|指定した部分文字列が文字列内に見つかったときに、ほかの文字列に置き換える|
|strip, rstrip, lstrip|空白文字を取り除く。stripは両端、rstripは右端のみ、lstripは左端のみから取り除く|
|split|指定した区切り文字によって文字列を複数に分割し、リストにして返す|
|lower|文字列内のアルファベット文字を小文字に変換する|
|upper|文字列内のアルファベット文字を大文字に変換する|
|casefold|文字列内のアルファベット文字を小文字に変換する。その際に、地域固有の変換文字は、同等の標準アルファベット文字での表現に置き換える|
|ljust, rjust|文字列が指定した幅以上になるように空白を詰める。その際に、文字列をそれぞれ左揃えと右揃えにする。反対側の余ったスペースは空白文字(デフォルト)で埋める|

## 7.3.2  正規表現

正規表現に用いるreモジュールの関数は、パターンマッチング、文字列置換、文字列分割の3つのカテゴリに分類できる  

In [102]:
import re

In [103]:
text = "foo    bar\t baz  \tqux"

In [104]:
re.split("\s+", text)

['foo', 'bar', 'baz', 'qux']

reモジュールのsplit関数を用いて、第一引数に当てはまる部分文字列を基準に分割したリストにできる  
reモジュールの関数として使った場合は第一引数に渡された文字列がコンパイル(高水準言語から低水準言語に変換すること)されてから、第二引数に当てはまる文字列を検索する

In [105]:
regex = re.compile("\s+")
regex.split(text)

['foo', 'bar', 'baz', 'qux']

正規表現のコンパイルは、reモジュールのcompile関数を用いて自分で明示的に実行しておくことができる  
返ってくる正規表現オブジェクトを用いることで、複数回同じ正規表現を行う場合は処理が1回で済む分早くなる

In [106]:
regex.findall(text)

['    ', '\t ', '  \t']

正規表現にマッチした文字列のリストを得たい場合は、findallメソッドを使う

In [107]:
text = """Dave Dave@google.com
Steve steve@gmail.com
Rob rob@gmail.com
Ryan ryan@yahoo.com
"""
pattern = r"[A-Z0-9._%+-]+@[A-Z0-9.-]+\.[A-Z]{2,4}"

regex = re.compile(pattern, flags=re.IGNORECASE)

In [108]:
regex.findall(text)

['Dave@google.com', 'steve@gmail.com', 'rob@gmail.com', 'ryan@yahoo.com']

上はfindallメソッドですべてのマッチした結果をリストに入れたものになる  
flagsに渡しているre.IGNORECASEは大文字をと小文字を区別しないようにするためのフラグ(re.Iでも可)

In [109]:
m = regex.search(text)
m

<re.Match object; span=(5, 20), match='Dave@google.com'>

In [110]:
m.group()

'Dave@google.com'

In [111]:
text[m.start(): m.end()]

'Dave@google.com'

searchメソッドだとマッチした最初のマッチオブジェクトを返す  

In [112]:
regex.match(text) == None

True

matchメソッドだと先頭が指定したパターンとマッチした場合のみマッチオブジェクトを返す

In [113]:
print(regex.sub("REDACTED", text))

Dave REDACTED
Steve REDACTED
Rob REDACTED
Ryan REDACTED



subメソッドは、パターンが見つかったすべての部分を、指定した別の文字列に置換して返す

In [114]:
pattern = r"([A-Z0-9._%+-]+)@([A-Z0-9._]+)\.([A-Z]{2,4})"

In [115]:
regex = re.compile(pattern, flags=re.I)

In [116]:
m = regex.match("wesm@bright.net")
m.groups()

('wesm', 'bright', 'net')

マッチしたパターンの文字列をさらに細かく分類したいときは、正規表現の表現内で丸カッコを用いてくくることで、検索結果をグループで分けくれる  
マッチオブジェクトに対してgroupsメソッドを呼ぶことで、グループに分けたタプルにして返してくれる

In [117]:
regex.findall(text)

[('Dave', 'google', 'com'),
 ('steve', 'gmail', 'com'),
 ('rob', 'gmail', 'com'),
 ('ryan', 'yahoo', 'com')]

グループを使った正規表現の場合、findallメソッドを使うとタプルのリストとして返される

In [118]:
print(regex.sub(r"Username: \1, Domain: \2, Suffix: \3", text))

Dave Username: Dave, Domain: google, Suffix: com
Steve Username: steve, Domain: gmail, Suffix: com
Rob Username: rob, Domain: gmail, Suffix: com
Ryan Username: ryan, Domain: yahoo, Suffix: com



subメソッドでも、正規表現全体にマッチしたそれぞれの文字列内のそれぞれのグループにアクセスすることができる  
アクセスには\1や\2のような特殊な表記方法を用いる  
\1やという表記は1番目のグループにマッチした文字列、\2は2番目のグループにマッチした文字列という風な意味になる  

正規表現のメソッド

|メソッド|説明|
|:-|:-|
|findall|指定した文字列に対してパターンマッチングを行い、見つかったパターンから位置がオーバーラップしないものすべてをリストにして返す|
|finditer|findallと同じだが、イテレータを返す|
|match|指定した文字列の先頭でパターンマッチングを行い、パターンにマッチした部分を、必要に応じてグループに分割する。パターンにマッチした場合はマッチオブジェクトを返し、マッチしていない場合はNoneを返す|
|search|指定した文字列がパターンにマッチしているか、文字列全体を調べ、マッチしている場合はマッチオブジェクトを返す。matchメソッドとは違い、マッチする場所は先頭のみにかかわらずどこでもよい|
|split|パターンにマッチした文字列で、指定した文字列を複数に分割し、リストにして戻す|
|sub, subn|指定した文字列に対してパターンマッチングを行い、見つかったパターンすべて(sub)、もしくは最初のn個だけ(subn)を、指定した置換文字列で置き換える。置換文字列内では、\1、\2などの表記を使用すれば、各グループに対してマッチした部分文字列を含めることができる|

## 7.3.3  pandasにおける文字列関数のベクトル化

乱雑なデータセットを分析できるようにクリーニングする際には、しばしば、大量の文字列処理と正則化が必要になる  
厄介なことに、文字列を含む列のデータが欠損しているために場合分けが必要になることがある

In [119]:
data = {"Dave": "dave@google.com", "Steve": "steve@gmail.com", "Rob": "rob@gmail.com", "Wes": np.nan}

In [120]:
data = pd.Series(data)
data

Dave     dave@google.com
Steve    steve@gmail.com
Rob        rob@gmail.com
Wes                  NaN
dtype: object

In [121]:
data.isnull()

Dave     False
Steve    False
Rob      False
Wes       True
dtype: bool

文字列操作メソッドや正規表現メソッドをデータ内のそれぞれの値に適用したい場合、mapメソッドを使って(lambda関数などの関数を渡すことで)実現できる  
しかしその場合、欠損値にあたると失敗してしまうという問題がある  
この問題をうまく対処できるようにするために、シリーズには欠損値を飛ばして処理してくれる配列指向の文字列操作メソッドがある  
これらのメソッドには、シリーズのstr属性からアクセスすることができる

In [122]:
data.str.contains("gmail")

Dave     False
Steve     True
Rob       True
Wes        NaN
dtype: object

上のcontainsメソッドはそれぞれの要素に指定した文字列が含まれているかを調べてくれる

In [123]:
pattern = r"([A-Z0-9._%+-]+)@([A-Z0-9._]+)\.([A-Z]{2,4})"
data.str.findall(pattern, flags=re.I)

Dave     [(dave, google, com)]
Steve    [(steve, gmail, com)]
Rob        [(rob, gmail, com)]
Wes                        NaN
dtype: object

str属性のメソッドにも正規表現を使うこともできる  
その場合は、re.IGNORECASEなどの正規表現オプションを指定することもできる

In [124]:
matches = data.str.findall(pattern, flags=re.I).str[0]
matches

Dave     (dave, google, com)
Steve    (steve, gmail, com)
Rob        (rob, gmail, com)
Wes                      NaN
dtype: object

In [125]:
matches.str.get(1)

Dave     google
Steve     gmail
Rob       gmail
Wes         NaN
dtype: object

ベクトル化された要素を得る方法はいくつかあり、str.getメソッドを方法と、str属性内のインデックスを指定する方法がある  

In [126]:
data.str[:5]

Dave     dave@
Steve    steve
Rob      rob@g
Wes        NaN
dtype: object

同様にstr属性にスライスを記法を使うことで、指定した範囲の文字列を切り出すことができる

In [127]:
data.str.extract(pattern, flags=re.I)

,0,1,2
Dave,dave,google,com
Steve,steve,gmail,com
Rob,rob,gmail,com
Wes,NaN,NaN,NaN


str属性のextractメソッドを使うことで、正規表現のグループごとにマッチした文字列をデータフレームとして取得することができる

ベクトル化された文字列メソッドの一部

|メソッド|説明|
|:-|:-|
|cat|文字列要素ごとに、(区切り文字を指定した場合はその文字を挟んで)連結する|
|contains|それぞれの文字列要素にパターンや正規表現にマッチする部分が含まれるかどうかを表す真偽値の配列を返す|
|count|パターンにマッチした回数を返す|
|endswith|それぞれの要素に対してx.endswith(pattern)を実行するのと同じ|
|startswith|それぞれの要素に対してx.startswith(pattern)を実行するのと同じ|
|findall|それぞれの文字列に対しおいて、指定したパターンや正規表現にマッチしたものすべてのリストにして返す|
|get|それそれの要素をインデックスで取得する(i番目の要素を返す)|
|isalnum|組み込みのstr.isalnumと同じ|
|islpha|組み込みのstr.isalphaと同じ|
|isdecimal|組み込みのstr.isdecimalと同じ|
|islower|組み込みのstr.islowerと同じ|
|isnumeric|組み込みのstr.isnumericと同じ|
|isupper|組み込みのstr.isupperと同じ|
|join|シリーズ内のそれぞれの文字列要素を、渡された区切り文字で結合して返す|
|len|それぞれの文字列の長さを返す|
|lower, upper|アルファベットの小文字・大文字の変換を行う|
|match|渡された正規表現をそれぞれの要素に対して用いてre.matchによるマッチングを行い、マッチしたかどうかを真偽値で返す|
|extract|パターンにグループを含む正規表現を用いて、それぞれの文字列から、各グループにマッチした部分文字列を(マッチした場合は)抽出し、そのグループに紐づけて返す。結果は1つのグループを1つの列とするデータフレームとなる|
|pad|文字列の左端や右端、両端に空白文字を追加する|
|center|pad(side="both")と同じ動作をする|
|repeat|文字列を複数回繰り返した文字列を戻す|
|replace|パターンや正規表現をにマッチした部分が文字列内に見つかった場合に、他の文字列に置き換える|
|slice|シリーズ内のそれぞれの文字列の一部を切り出す|
|split|区切り文字や正規表現で文字列を分割する|
|strip, rstrip, lstrip|文字列の両端・右端・左端から空白文字を取り除く|